In [1]:
# --- CELL 1: SETUP ---
# 1. Uninstall the conflicting library first
!pip uninstall -y -q tensorflow

# 2. Install the specific versions that work on Kaggle GPU
!pip install -q --upgrade pip
!pip install -q transformers==4.40.0 accelerate==0.29.3 datasets==2.19.0 jiwer soundfile librosa

import os
import glob
import pandas as pd
import torch
from transformers import pipeline
from tqdm.notebook import tqdm

# 3. Check GPU
# If this still says CPU, toggle the Accelerator in settings to 'None' and back to 'GPU P100'
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to: {'GPU' if device == 0 else 'CPU'}")


Device set to: GPU


In [2]:
# Model ID
MODEL_ID = "kaarthu2003/wav2vec2-IEEEAccess-FinalRun-4Datasets"

# Load the pipeline
pipe = pipeline(
    "automatic-speech-recognition", 
    model=MODEL_ID, 
    device=device 
)

print("Model loaded successfully.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/886 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Model loaded successfully.


In [3]:
pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 262.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 210.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [kaggle]11/13 [black]sdk]
Note: you may need to restart the kernel to use updated packages.


In [4]:

# Setup Kaggle credentials
import os
import json

# Ensure the directory exists
!mkdir -p ~/.kaggle

# Move the uploaded kaggle.json to the required location
!mv /workspace/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the specific dataset you requested
# URL: https://www.kaggle.com/datasets/anvitamanne/telugu-noisy-data
!kaggle datasets download -d anvitamanne/telugu-noisy-data --unzip -p /workspace/data

mv: cannot stat '/workspace/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/anvitamanne/telugu-noisy-data
License(s): unknown
100%|█████████████████████████████████████▉| 1.62G/1.63G [00:49<00:00, 31.1MB/s]
100%|██████████████████████████████████████| 1.63G/1.63G [00:49<00:00, 35.0MB/s]


In [5]:
# Check what's inside the data folder
print("--- Root Folders ---")
!ls -F /workspace/data

# Find the location of the CSV file (Ground Truth)
print("\n--- Finding Metadata CSV ---")
!find /workspace/data -name "*.csv"

--- Root Folders ---
'archive (1)'/	'archive (2)'/	'archive (3)'/

--- Finding Metadata CSV ---
/workspace/data/archive (3)/noisy_dataset/transcriptions.csv
/workspace/data/archive (2)/indictts_noise_transcripts.csv
/workspace/data/archive (1)/mozilla_transcripts_csv/transcripts.csv


In [6]:
# Install ffmpeg on the RunPod system
!apt-get update && apt-get install -y ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2248 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:8 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]     
Get:9 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/ma

In [7]:

# Setup Kaggle credentials
import os
import json

# Ensure the directory exists
!mkdir -p ~/.kaggle

# Move the uploaded kaggle.json to the required location
!mv /workspace/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the specific dataset you requested
# URL: https://www.kaggle.com/datasets/anvitamanne/telugu-noisy-data
!kaggle datasets download -d anvitamanne/telugu-noisy-data --unzip -p /workspace/data

mv: cannot stat '/workspace/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/anvitamanne/telugu-noisy-data
License(s): unknown
100%|█████████████████████████████████████▉| 1.63G/1.63G [00:45<00:00, 40.0MB/s]
100%|██████████████████████████████████████| 1.63G/1.63G [00:45<00:00, 38.5MB/s]


In [8]:
# Check what's inside the data folder
print("--- Root Folders ---")
!ls -F /workspace/data

# Find the location of the CSV file (Ground Truth)
print("\n--- Finding Metadata CSV ---")
!find /workspace/data -name "*.csv"

--- Root Folders ---
'archive (1)'/	'archive (2)'/	'archive (3)'/

--- Finding Metadata CSV ---
/workspace/data/archive (3)/noisy_dataset/transcriptions.csv
/workspace/data/archive (2)/indictts_noise_transcripts.csv
/workspace/data/archive (1)/mozilla_transcripts_csv/transcripts.csv


In [9]:
# Install ffmpeg on the RunPod system
!apt-get update && apt-get install -y ffmpeg

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (178 kB/s)                          
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 126 not upgraded.


In [10]:
# Verify ffmpeg version
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [11]:
import os
import glob
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm

# Mapping to the specific folders you discovered on RunPod
DATASET_PATHS = {
    "noisy_dataset": "/workspace/data/archive (3)/noisy_dataset",
    "indictts_noise": "/workspace/data/archive (2)",
    "mozilla_noise": "/workspace/data/archive (1)"
}

results = []
limit = 5  # Set to None to run ALL files

print(f"Starting inference (Test Limit: {limit})...")

for dataset_name, base_path in DATASET_PATHS.items():
    print(f"\nProcessing {dataset_name}...")
    
    # Find all audio files in the current folder
    files = []
    for ext in ['*.wav', '*.mp3', '*.flac']:
        files.extend(glob.glob(os.path.join(base_path, '**', ext), recursive=True))
    
    print(f"  Found {len(files)} files.")
    
    # Apply limit
    files_to_process = files[:limit] if limit else files
    
    for audio_path in tqdm(files_to_process, desc=f"Inference on {dataset_name}"):
        try:
            # Predict
            output = pipe(audio_path, generate_kwargs={"language": "telugu"})
            
            results.append({
                "dataset": dataset_name,
                "filename": os.path.basename(audio_path),
                "prediction": output["text"]
            })
        except Exception as e:
            print(f"Error on {os.path.basename(audio_path)}: {e}")

# Save and Display
if results:
    preds_df = pd.DataFrame(results)
    preds_df.to_csv("whisper_predictions.csv", index=False)
    print(f"\nSaved {len(results)} predictions to 'whisper_predictions.csv'")
    
    # Force full display for Telugu text clarity
    pd.set_option('display.max_colwidth', None)
    print("\n--- PREDICTION PREVIEW ---")
    display(preds_df.head(10)) 
else:
    print("No predictions generated. Double check if the audio files are inside the archive folders.")

Starting inference (Test Limit: 5)...

Processing noisy_dataset...
  Found 3908 files.


Inference on noisy_dataset:   0%|          | 0/5 [00:00<?, ?it/s]


Processing indictts_noise...
  Found 1149 files.


Inference on indictts_noise:   0%|          | 0/5 [00:00<?, ?it/s]


Processing mozilla_noise...
  Found 2311 files.


Inference on mozilla_noise:   0%|          | 0/5 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Saved 15 predictions to 'whisper_predictions.csv'

--- PREDICTION PREVIEW ---


,dataset,filename,prediction
0,noisy_dataset,set2_tem_09584_02077832949.wav,అవన్నీ నాకు తెలుసు
1,noisy_dataset,set2_tem_09584_02047229027.wav,ఇవి ఐదు వర్గములుగా విభజింపబడిందవి
2,noisy_dataset,set2_tem_09584_02021897210.wav,అలాంటి సమయంలో
3,noisy_dataset,set2_tem_09584_01629319103.wav,జనసంచారం ఉన్న ప్రాంతాలలో వ్యభిచారం నేరు
4,noisy_dataset,set2_tem_09584_01489049719.wav,గుంతలో తేమఎప్పుడూ తగినతు ఉండాలి
5,indictts_noise,noisy_telugu_001148.wav,కోడంగల్ కోస్గి నారాయణపేట నుంచి బస్సు సౌకర్యం ఉంది
6,indictts_noise,noisy_telugu_001147.wav,బిజ్ఞానశాస్త్ర విషయాలకు సరైన వర్గీకరణ ఇంకా జరగలేదు
7,indictts_noise,noisy_telugu_001146.wav,మీరు అప్లోడ్ చేసిన ఫోటో బాగుంది
8,indictts_noise,noisy_telugu_001145.wav,మన ఊరి పాండవులు ఒక మంచి తెలుగు సినిమా
9,indictts_noise,noisy_telugu_001144.wav,వ్యాసం పేరు హిఠాపురం నాగేశ్వరరావు అని మార్చాలి


In [12]:
!pip install -q jiwer

In [13]:
import pandas as pd
import os
import glob

# Base path where you unzipped the data
BASE_DIR = "/workspace/data"

# Mapping the specific locations based on your screenshots
MAPPING_CONFIG = {
    "mozilla": {
        "csv": os.path.join(BASE_DIR, "archive (1)/mozilla_transcripts_csv/transcripts.csv"),
        "audio_dir": os.path.join(BASE_DIR, "archive (1)/mozilla_noise")
    },
    "indictts": {
        "csv": os.path.join(BASE_DIR, "archive (2)/indictts_noise_transcripts.csv"),
        "audio_dir": os.path.join(BASE_DIR, "archive (2)/indictts_noise_audio")
    },
    "openslr": {
        "csv": os.path.join(BASE_DIR, "archive (3)/noisy_dataset/transcriptions.csv"),
        "audio_dir": os.path.join(BASE_DIR, "archive (3)/noisy_dataset/clips")
    }
}

master_data = []

for label, paths in MAPPING_CONFIG.items():
    if os.path.exists(paths['csv']):
        df = pd.read_csv(paths['csv'])
        df.columns = [c.lower().strip() for c in df.columns]
        
        # Identify columns
        file_col = next((c for c in df.columns if any(x in c for x in ['file', 'path', 'audio'])), None)
        text_col = next((c for c in df.columns if any(x in c for x in ['text', 'sentence', 'transcript'])), None)
        
        for _, row in df.iterrows():
            # Get just the filename (e.g., 'audio123.wav')
            fname = os.path.basename(str(row[file_col]))
            # Construct the REAL path on your RunPod disk
            full_audio_path = os.path.join(paths['audio_dir'], fname)
            
            if os.path.exists(full_audio_path):
                master_data.append({
                    "dataset": label,
                    "audio_path": full_audio_path,
                    "reference": str(row[text_col])
                })

# Convert to a single clean DataFrame
final_df = pd.DataFrame(master_data)
print(f" Success! Found {len(final_df)} valid audio-text pairs across all archives.")
print(final_df.groupby('dataset').size())

 Success! Found 7368 valid audio-text pairs across all archives.
dataset
indictts    1149
mozilla     2311
openslr     3908
dtype: int64


In [14]:
sample = final_df.sample(1).iloc[0]
print(f"Testing Path: {sample['audio_path']}")
print(f"Reference Text: {sample['reference']}")

# Try a single inference to confirm
out = pipe(sample['audio_path'], generate_kwargs={"language": "telugu"})
print(f"Whisper Prediction: {out['text']}")

Testing Path: /workspace/data/archive (1)/mozilla_noise/common_voice_te_40067678_noisy.wav
Reference Text: సూక్తిలేని మాట శ్రుతిలేని పాటరా
Whisper Prediction: సూక్తిలేని మాట శ్రుతిలేని పాటరా


In [15]:
import pandas as pd
import os
import re
from jiwer import wer, cer
from tqdm.notebook import tqdm

# --- 1. SETUP & NORMALIZATION ---
def normalize_telugu(text):
    if not isinstance(text, str): return ""
    # Strip punctuation to prevent artificial WER inflation
    text = re.sub(r'[^\u0C00-\u0C7F\s]', '', text) 
    return re.sub(r'\s+', ' ', text).strip()

individual_results = []

# --- 2. FULL EXECUTION ---
# Using 'final_df' which contains all 7,368 mapped pairs
print(f"Starting Full Evaluation of {len(final_df)} files...")

for idx, row in tqdm(final_df.iterrows(), total=len(final_df)):
    try:
        # Run Whisper Inference
        out = pipe(row['audio_path'], generate_kwargs={"language": "telugu"})
        
        # Clean both for fair comparison
        ref_clean = normalize_telugu(row['reference'])
        hyp_clean = normalize_telugu(out["text"])
        
        individual_results.append({
            "dataset": row['dataset'],
            "reference": ref_clean,
            "prediction": hyp_clean,
            "wer": wer(ref_clean, hyp_clean) if ref_clean else 0,
            "cer": cer(ref_clean, hyp_clean) if ref_clean else 0
        })
    except Exception as e:
        continue

# --- 3. DISPLAY INDIVIDUAL REPORTS ---
res_df = pd.DataFrame(individual_results)
pd.set_option('display.max_colwidth', None)

for ds in res_df['dataset'].unique():
    subset = res_df[res_df['dataset'] == ds]
    print(f"\n{'='*30} {ds.upper()} FULL REPORT {'='*30}")
    print(f"Total Samples: {len(subset)}")
    print(f"AVG WER: {subset['wer'].mean():.4f} ({subset['wer'].mean()*100:.2f}%)")
    print(f"AVG CER: {subset['cer'].mean():.4f} ({subset['cer'].mean()*100:.2f}%)")
    print("\nTOP 10 TRANSCRIPTS:")
    display(subset[['reference', 'prediction', 'wer']].head(10))

Starting Full Evaluation of 7368 files...


  0%|          | 0/7368 [00:00<?, ?it/s]


============================== MOZILLA FULL REPORT ==============================
Total Samples: 2311
AVG WER: 0.1165 (11.65%)
AVG CER: 0.0302 (3.02%)

TOP 10 TRANSCRIPTS:


,reference,prediction,wer
0,అభ్యర్థులు పరీక్ష కేంద్రాల్లో రిపోర్ట్ చేయాలి,అభ్యర్థులు పరీక్ష కేంద్రాల్లో రిపోర్ట్ చేయాలి,0.000000
1,గాలి వీచు చెట్లు పూలుపూచు,గాలి వీచు చెట్లు పూలుపూచు,0.000000
2,శుక్రవారం వారు ఇక్కడ మీడియాతో మాట్లాడారు,శుక్రోారం వారు ఇక్కడ మీడియాతో మాట్లాడారు,0.200000
3,చదువురాని మొద్దు సభల రాణింపదు,చదువురాని మొద్దు సభల రాణింపదు,0.000000
4,చెడ్డవాని చెలిమి చిచ్చువోలె,చెడ్డవాని చెలిమి చిచ్చువోలె,0.000000
5,సాధుపుంగవులకు జగమే కుటుంబంబు,సాధుపుంగవులకు జగమే కుటుంబంబు,0.000000
6,విద్య గనని జనులు,విద్య గననిజనులు,0.666667
7,జిల్లా మొత్తం వర్షాలు పడ్డాయి,జిల్లా మొత్తం వర్షాలు పడ్డాయి,0.000000
8,మదగజమ్ము వీడి,మదగజమ్ము వీడి,0.000000
9,బద్దకంపు మొద్దు బల్లపై నెక్కెరా,మద్దకంపు మొద్దు బల్లపై నెక్కెరా,0.250000



============================== INDICTTS FULL REPORT ==============================
Total Samples: 1149
AVG WER: 0.2498 (24.98%)
AVG CER: 0.0480 (4.80%)

TOP 10 TRANSCRIPTS:


,reference,prediction,wer
2311,ఆయనను ఇక్కడికి తీసుకురావటం దేనికమ్మా నా కాయనతో పనిలేదు అన్నాడు మిత్రవిందకుడు,ఆయనను ఇక్కడికి దీసుకురావటం దేనికమ్మా నా కాయనతో పనిలేదు అన్నాడు మిత్రవిందకుడు,0.111111
2312,బాబూ ఇది పద్మం కాదు అసిధారా చక్రం అన్నాడు నరకవాసి,బాబూ ఇది పద్మం కాదు అసిధారా చక్రం అన్నాడు నరకవాసి,0.000000
2313,చూశావా నా కివ్వటం ఇష్టంలేక ఆ మాట అంటున్నావు అన్నాడు మిత్రవిందకుడు,చూశావా నా కివ్వటం ఇష్టంలేక ఈ మాట అంటున్నావు అన్నాడు మిత్రవిందకుడు,0.111111
2314,అప్పుడు ఇంద్రుడు మిత్రవిందకుడికి ఈ విధంగా జవాబు చెప్పాడు,అప్పుడు ఇంద్రుడు మిత్రవిందకుడికి ఈ విధంగా జవాబు చెప్పాడు,0.000000
2315,బాబు చంపేశారు అంటూ ధర్మ గుప్తుడు ఆ మనిషి కేసి వెళ్ళాడు,బాబు చంపేశారు అంటూ ధర్మగుత్తుడు మనిషికేసి వెళ్ళాడు,0.555556
2316,దొంగలు తల పగలగొట్టారు అంటూ ధర్మగుప్తుడు ప్రాణాలు వదిలాడు,దొంగలు తలపగలగొట్టారు అంటూ ధర్మగుప్తుడు ప్రాణాలు వదిలాడు,0.285714
2317,నీకు వచ్చిన లోటేమిటి అన్ని ఏర్పాట్లు అయ్యాక పెళ్ళి ఎట్లా ఆగుతుంది నన్ను వేధించకు అన్నాడు లోహితుడు,నీకు వచ్చిన లోటేమిటి అన్నీ ఏర్పాట్లు అయ్యాక పెళ్లిఏట్ల ఆగుతుంది నన్ను వేహించకు అన్నాడు లోహిదుు,0.384615
2318,ఏదీ నువ్వు కాస్త సహాయంచెయ్యి తోక పట్టుకు లాగు అన్నది ఆవు,ఏదీ నువ్వు కాత సహాయంచెయ్యి తోక పట్టుకు లాగు అన్నదిఆవు,0.333333
2319,ఆవు అత్తా ఏమిటి అడావుడి అని అడిగింది కుందేలు,ఆవు అత్తా ఏమిటి అడావుడి అని అడిగింది కుందేలు,0.000000
2320,అంతా విని బంధువులు ఎంత అవమానం ఎంత అప్రతిష్ఠ అని ముక్కుల మీద వేళ్ళు వేసుకున్నారు,అంతా వీని బంధవులు ఎంతఅవమానం ఎంత అప్్రతిస్ట అని ముక్కులమీద వేళ్లువేసుకున్నారు,0.750000



============================== OPENSLR FULL REPORT ==============================
Total Samples: 3908
AVG WER: 0.1840 (18.40%)
AVG CER: 0.0393 (3.93%)

TOP 10 TRANSCRIPTS:


,reference,prediction,wer
3460,చదవక తప్పదన్న మాట,చదవక తప్పదన్న మాట,0.000000
3461,సాలీనా దీని ఉత్పాదన ఇరవై టన్నులకి మించదు,సాలీనా దీని ఉత్పాదన ఇరవై టన్నులకి మించదు,0.000000
3462,అదే అతడి సార్ధక నామధేయం అయింది,అదే అతడి సార్ధిక నామధేయం అయయింది,0.400000
3463,కూడా పూర్తి చేశాడు,కూడా పూర్తి చేశాడు,0.000000
3464,దీనికి పెద్దగా స్పందన రాలేదు,దీనికి పెద్దగా స్పందన రాలేదు,0.000000
3465,దుర్మార్గుల రక్తంతో భూమి తడుసిపోతుంది,దుర్మార్గుల రక్తంతో భూమి తడుసిపోతుంది,0.000000
3466,నేను కూడా ఒక హామ్ వ్యాసాన్ని రాసాను,లేనికోడే ఒక హామ్ వ్యాసాన్ని రాసాను,0.333333
3467,ఈ సినిమాల జాబితా పెట్టింది నేనే,ఈ సినిమల జాగితా కుత్టిందునాను,0.800000
3468,చీలమండ కాలు చివరి భాగము,చీలమండ కాలు చివరి భాగుననం,0.250000
3469,అనగా మన దేశానికి హిందీ అధికార భాష,అనగా మన దేశానికి ముంది అధికార భాష,0.166667


In [16]:
from jiwer import wer, cer
import pandas as pd

# 1. CORPUS-LEVEL CALCULATION
# We treat the entire 7k+ file dataset as one continuous block
all_refs = res_df['reference'].tolist()
all_hyps = res_df['prediction'].tolist()

# Global metrics: This does NOT average individual scores
total_corpus_wer = wer(all_refs, all_hyps)
total_corpus_cer = cer(all_refs, all_hyps)

# 2. FORMATTED SUMMARY TABLE
summary_df = pd.DataFrame({
    "Total Files Processed": [len(res_df)],
    "Global Corpus WER": [f"{total_corpus_wer:.4f} ({total_corpus_wer*100:.2f}%)"],
    "Global Corpus CER": [f"{total_corpus_cer:.4f} ({total_corpus_cer*100:.2f}%)"]
})

print("\n" + "="*60)
print("      FINAL COMBINED EVALUATION (ONE PIECE CORPUS METRICS)")
print("="*60)
display(summary_df)

# 3. 10 SAMPLE TRANSCRIPTS FROM COMBINED DATA
# Displaying 10 random samples from the total processed pool
print("\n10 SAMPLE TRANSCRIPTS FROM THE COMBINED DATASET:")
pd.set_option('display.max_colwidth', None)
display(res_df.sample(10)[['dataset', 'reference', 'prediction']])

# 4. SAVE FINAL OUTPUT
# Save the full results locally to /workspace
res_df.to_csv("/workspace/whisper_full_corpus_results.csv", index=False)
print(f"\n✅ Full evaluation saved to: /workspace/whisper_full_corpus_results.csv")


      FINAL COMBINED EVALUATION (ONE PIECE CORPUS METRICS)


,Total Files Processed,Global Corpus WER,Global Corpus CER
0,7368,0.1959 (19.59%),0.0458 (4.58%)



10 SAMPLE TRANSCRIPTS FROM THE COMBINED DATASET:


,dataset,reference,prediction
5658,openslr,జనాభా మరియు సమాజ జీవ శాస్త్రా సిరీస్ పదహారు,జనాభా మరియు సమాజ జీవ శాస్త్రా సిరీ పదహారు
5409,openslr,వక్రత భావ సంభంధమైనది,వక్రత భావ సంభంధమైనది
2043,mozilla,ప్రవాహ వేగం స్వల్పంగా తగ్గింది,ప్రవాహ వేగం స్వల్పంగా తగ్గింది
6831,openslr,దాని బాయిలర్ లో దోషం కనబడింది,దాని బాయిలర్ లో దోషం కనబడింది
908,mozilla,కలిమి గలుగ నేస్తకాండ్రు వేలకు వేలు,కలిమి గలుగ నేస్తకాండ్రు వేలకు వేలు
3116,indictts,వారి కలయిక మిగతా చిత్ర కధ,వారి కలయిక మిగతా చిత్ర కధ
2345,indictts,రాజు మరింత బిర్రబిగిసి నా నోటంట వచ్చినమాట జరిగితీరవలసిందే అన్నాడు,రాజు మరింత బిర్రబిగిినా నోటంట వచ్చిన మాట జరిగిీరవలసిందే అన్నాడు
4827,openslr,వేదజననీ వేదములకు తల్లి,వేదజననీ వేదందులకి తల్లి
7282,openslr,ఆచమనం చేసిన తరువాత నోటిని తుడుచుకోవాలి,ఆచమనం చేసిన తరువాత నోటిని తుడుచుకోవాి
6474,openslr,పశ్చిమ గోదావరి జిల్లా భారత దేశము యొక్క ఆంధ్ర ప్రదేశ్ రాష్ట్రములోని ఒక జిల్లా,పశ్చిమ గోదావరి జిల్లా భారతదేశము యొక్క ఆంధ్ర ప్రదేశ్ రాష్ట్రములోని ఒక జిల్లా



✅ Full evaluation saved to: /workspace/whisper_full_corpus_results.csv


In [17]:
import pandas as pd

# 1. Ensure columns are wide enough to read the Telugu text
pd.set_option('display.max_colwidth', None)

# 2. List the specific indices from your previous reference image
target_indices = [3294, 2628, 122, 4592, 224, 3448, 4827, 5311, 4362, 5388]

# 3. Pull those exact rows using .loc[]
# This ensures you are looking at the exact same sentences
comparison_samples = res_df.loc[target_indices, ['dataset', 'reference', 'prediction']]

print("--- DIRECT COMPARISON: PREVIOUS SAMPLES VS NEW PREDICTIONS ---")
display(comparison_samples)

# 4. Optional: Save these specific comparisons to see if WER improved
comparison_samples.to_csv("/workspace/direct_comparison_samples.csv", index=True)

--- DIRECT COMPARISON: PREVIOUS SAMPLES VS NEW PREDICTIONS ---


,dataset,reference,prediction
3294,indictts,రెండు కి సంబంధించిన తెలుగు మాటలు జోడీజోడుజతజంట ఉమ్మడి,రెండుకి సంబంధించిన తెలుగుమాటలు జోడిజోడు జత జంట ఉమ్మడి
2628,indictts,రాజు అలా అనేసరికి సునందుడు చేసేది లేక తలవంచుకొని ఇంటికి వెళ్లిపోయాడు,రాజు అలా అనేసరికి సునందుడు చేసేది లేక తలవంచుకొని ఇంటికి వెళ్లిపోయాడు
122,mozilla,వివిధ సంఘాలు మద్దతు ప్రకటించాయి,వివిధ సంఘాలు మద్దతు ప్రకటించాయి
4592,openslr,నటుడిగా సఫలత పొందలేదు,నటుడిగా సఫలత పొందలేదు
224,mozilla,మధుకరంబు వచ్చి మకరందమును ద్రావు,మధుకరంబు వచ్చి మకరందమును ద్రావు
3448,indictts,ఆంధ్ర ప్రదేశ్ రాష్ట్రానికి చెందిన నగరము మరియు అదే పేరుగల జిల్లా యొక్క ముఖ్య పట్టణము,ఆంధ్ర ప్రదేశ్ రాష్ట్రానికి చెందిన నగరము మరియు అదేపేరుగల జిల్లా యొక్క ముఖ్య పట్టణం
4827,openslr,వేదజననీ వేదములకు తల్లి,వేదజననీ వేదందులకి తల్లి
5311,openslr,వీటిలో పగలు కూడ జరిగే ఘట్టాలు మూడు,వీటిలో పదలు కూడ జరిగే ఘట్టాలు మూడు
4362,openslr,మధ్యధరా ప్రాంతంచే చుట్టి వున్నది,మధ్యధరా ప్రాంతంచే చుట్టివున్నది
5388,openslr,చట్టాలు దేశాలవారీగా మారుతుంటాయి,చట్టాలు దేశాలవారిగా మారుతుంతాయి
